In [ ]:
import copy
import json

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import requests
import seaborn as sns
import spectrum_utils.spectrum as sus
import spectrum_utils.plot as sup
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='serif')
sns.set_palette(['#9e0059', '#6da7de', '#ee266d', '#dee000', '#eb861e'])
sns.set_context('paper', font_scale=0.6)

In [ ]:
usis = [
    'mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB00003135669',
    'mzspec:MSV000082049:20_51.mzML:scan:106',
    'mzspec:MSV000085561:011c.mzML:scan:2864',
    'mzspec:MSV000085561:010c.mzML:scan:2829',
    'mzspec:MSV000085561:018b.mzML:scan:2609',
    'mzspec:MSV000082650:M031_48.mzML:scan:1501'
]
compounds = [
    ['Hexanoylcarnitine, C6:0', None],
    ['Hexanoylcarnitine – 12.036 Da', '3-Hydroxybutyrylcarnitine'],
    ['Hexanoylcarnitine – 2.016 Da', 'C6:1'],
    ['Hexanoylcarnitine + 5.955 Da', 'Benzoylcarnitine'],
    ['Hexanoylcarnitine + 15.995 Da', '3-Hydroxyhexanoylcarnitine'],
    ['Hexanoylcarnitine + 114.068 Da', 'Dodecanedioylcarnitine, C12-DC'],
]

In [ ]:
spectra = []
for usi in usis:
    spectrum_dict = json.loads(requests.get(
        f'https://metabolomics-usi.ucsd.edu/json/?usi1={usi}',
        timeout=None).text)
    mz, intensity = zip(*spectrum_dict['peaks'])
    spectra.append(sus.MsmsSpectrum(
        usi, spectrum_dict['precursor_mz'], 0, mz, intensity))

In [ ]:
sup.colors['C4H5O2'] = '#9e1d5c'
sup.colors['C7H14NO2'] = '#5a86b0'
sup.colors['TMAI'] = '#00a69e'
sup.colors['3-Hydroxy'] = '#ff38ba'

fig, axes = plt.subplots(len(spectra), 1, sharex=True, sharey=True,
                         figsize=(3.5, len(spectra) * 1))
axes = axes.ravel()

for i, (spec, ax) in enumerate(zip(spectra, axes)):
    spec = copy.copy(spec)
    spec.set_mz_range(min_mz=0, max_mz=400).remove_precursor_peak(1, 'Da')
    # Peak annotations.
    try:
        spec.annotate_mz_fragment(85.0284, 1, 0.1, 'Da', text='C$_4$H$_5$O$_2^+$')
    except ValueError:
        pass
    try:
        spec.annotate_mz_fragment(60.0808, 1, 0.1, 'Da', text='TMAI')
    except ValueError:
        pass
    try:
        spec.annotate_mz_fragment(144.1019, 1, 0.1, 'Da', text='C$_7$H$_{14}$NO$_2^+$')
    except ValueError:
        pass
    # Loss annotations.
    # TMA (trimethylamine) loss: 59.0735
    ax.axvspan(spec.precursor_mz - 59.0735, spec.precursor_mz,
               color='#5fa05c', alpha=0.35, lw=0)
    # C7H14NO2 (carnitine - H2O) loss: 144.1019
    ax.axvspan(spec.precursor_mz - 144.1019, spec.precursor_mz - 59.0735,
               color='#dddf1d', alpha=0.35, lw=0)
    # Carnitine loss: 161.1052
    ax.axvspan(spec.precursor_mz - 161.1052, spec.precursor_mz - 144.1019,
               color='#d1d4d4', alpha=0.35, lw=0)
    
    # 3-Hydroxy peak annotations.
    if i in (1, 4):
        spec_ins = copy.copy(spec).set_mz_range(143.5, 146.5)
        spec_ins.annotate_mz_fragment(144.1019, 1, 0.1, 'Da', text='')
        spec_ins.annotate_mz_fragment(145.0496, 1, 0.1, 'Da', text=' ▸')
        for annotation in spec_ins.annotation:
            if annotation is not None:
                if annotation.calc_mz == 144.1019:
                    annotation.ion_type = 'C7H14NO2'
                elif annotation.calc_mz == 145.0496:
                    annotation.ion_type = '3-Hydroxy'
                    
        axins = inset_axes(ax, width=0.4, height=0.2, loc='upper right')
        sup.spectrum(spec_ins, grid=False, ax=axins)
        axins.set_xlim(143, 147)
        axins.set_ylim(0, 1)
        axins.set_xlabel('')
        axins.set_ylabel('')
        axins.xaxis.set_major_locator(mticker.FixedLocator([143, 145, 147]))
        axins.yaxis.set_major_locator(mticker.FixedLocator([]))
        axins.minorticks_off()
        axins.xaxis.set_tick_params(size=2, pad=2, labelsize='x-small')
        sns.despine(ax=axins)
    # DC peak annotations.
    if i == 5:
        # Carnitine + H2O loss: 179.1205
        ax.axvspan(spec.precursor_mz - 179.1205, spec.precursor_mz - 161.1052,
                   color='#eb8722', alpha=0.35, lw=0)
        # Carnitine + carboxylic acid loss: 207.1296
        ax.axvspan(spec.precursor_mz - 207.1296, spec.precursor_mz - 179.1205,
                   color='#223f9b', alpha=0.35, lw=0)
    
    for annotation in spec.annotation:
        if annotation is not None:
            if annotation.calc_mz == 60.0808:
                annotation.ion_type = 'TMAI'
            elif annotation.calc_mz == 85.0284:
                annotation.ion_type = 'C4H5O2'
            elif annotation.calc_mz == 144.1019:
                annotation.ion_type = 'C7H14NO2'
    
    sup.spectrum(spec, grid=False, ax=ax)
    ax.set_xlim(0, 450)
    ax.set_ylim(0, 1)

for ax, compound, spec in zip(axes, compounds, spectra):
    ax.text(1, 1.4, f'{compound[0]}  ($m$/$z$ = {spec.precursor_mz:.3f})',
            fontsize='large', ha='center', math_fontfamily='dejavuserif',
            transform=ax.transAxes)
    if compound[1] is not None:
        ax.text(1, 1.15, f'→ {compound[1]}', fontsize='large', ha='center',
                math_fontfamily='dejavuserif', transform=ax.transAxes)

for ax in axes[:-1]:
    ax.set_xlabel('')
    
for ax in axes:
    sns.despine(ax=ax)
    
plt.tight_layout()
    
plt.savefig('acylcarnitine_suspects.png', bbox_inches='tight', dpi=300)
plt.show()
plt.close()